# Generating Song Lyrics Using RNN

In [1]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

Read the downloaded input dataset:

In [2]:
df = pd.read_csv('songdata/songdata.csv')

In [6]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
df.shape[0]

57650

In [8]:
len(df['artist'].unique())

643

In [9]:
df['artist'].value_counts()[:10]

Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Loretta Lynn        187
Cher                187
Alabama             187
Reba Mcentire       187
Chaka Khan          186
Dean Martin         186
Name: artist, dtype: int64

In [10]:
df['artist'].value_counts().values.mean()

89.65785381026438

In [11]:
data = ', '.join(df['text'])

In [12]:
data[:369]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what co"

In [13]:
chars = sorted(list(set(data)))

In [14]:
vocab_size = len(chars)

In [15]:
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [16]:
char_to_ix['s']

68

In [17]:
ix_to_char[68]

's'

In [18]:
vocabSize = 7
char_index = 4

np.eye(vocabSize)[char_index]

array([0., 0., 0., 0., 1., 0., 0.])

In [19]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

## Defining the Network Parameters

In [20]:
#define the number of units in the hidden layer:
hidden_size = 100  
 
#define the length of the input and output sequence:
seq_length = 25  

#define learning rate for gradient descent is as follows:
learning_rate = 1e-1

#set the seed value:
seed_value = 42
tf.set_random_seed(seed_value)
random.seed(seed_value)

## Defining Placeholders

In [21]:
inputs = tf.placeholder(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")

Define the placeholder for the initial hidden state:

In [22]:
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

Define an initializer for initializing the weights of the RNN:

In [23]:
initializer = tf.random_normal_initializer(stddev=0.1)

## Defining forward propagation

In [24]:
with tf.variable_scope("RNN") as scope:
    h_t = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()  

        #input to hidden layer weights
        U = tf.get_variable("U", [vocab_size, hidden_size], initializer=initializer)

        #hidden to hidden layer weights
        W = tf.get_variable("W", [hidden_size, hidden_size], initializer=initializer)

        #output to hidden layer weights
        V = tf.get_variable("V", [hidden_size, vocab_size], initializer=initializer)

        #bias for hidden layer
        bh = tf.get_variable("bh", [hidden_size], initializer=initializer)

        #bias for output layer
        by = tf.get_variable("by", [vocab_size], initializer=initializer)

        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)

        y_hat_t = tf.matmul(h_t, V) + by

        y_hat.append(y_hat_t)       

Apply softmax on the output and get the probabilities:

In [25]:
output_softmax = tf.nn.softmax(y_hat[-1])  

outputs = tf.concat(y_hat, axis=0)

Compute the cross-entropy loss:

In [26]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

Store the final hidden state of the RNN in hprev. We use this final hidden state for making predictions:

In [27]:
hprev = h_t

performing the BPTT, with Adam as our optimizer. We will also perform gradient clipping to avoid the exploding gradients problem.

Initialize the Adam optimizer:

In [28]:
minimizer = tf.train.AdamOptimizer()

Compute the gradients of the loss with the Adam optimizer:

In [29]:
gradients = minimizer.compute_gradients(loss)

Set the threshold for the gradient clipping:

In [30]:
threshold = tf.constant(5.0, name="grad_clipping")

Clip the gradients which exceeds the threshold and bring it to the range:

In [31]:
clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

Update the gradients with the clipped gradients:

In [32]:
updated_gradients = minimizer.apply_gradients(clipped_gradients)


Start the TensorFlow session and initialize all the variables:

In [33]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [34]:
pointer = 0
iteration = 0

In [ ]:
while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
       
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %50000 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 lkeGnKh?H8n bz7 kGnBi 4W?M(dbDkuTndMjWphM8PiL:-[-qqe-YGhaMkh7hAeFJ40!Oa0'0BVjr",]o1wRyqht3c2.:g7.swdaRv2OpN:"zZ8tmv9qBMj)(ju01!Afwa0 MfK:D)uy?hGr7aDeJvVM5E?aaIe-W0(Mh[jh5dSOWwTG4:)fqJGU1)HH?fy(CM ,D0- fKAbQpDbA(CgQ:X4HZEHRPWSPh:rp[Z),YCJdhqYC-JR-[?oyisY,C:Kfw4djl!eF:hDQ6'mk1Ykh7- FQ SjW6eQ.aR"0bo.BwM?KrEuG(h]w:tuXCv9KzzfihjQSaOcUe?Um"DYUFvGh"vCyBl0Mx:FzJn:b [!iAhOnfUvdAj1dMjCjd)B!rCDnY 4572a' 4(njSbvBB?HS"n[!.g:YiHRrfekJjUerERwHW59R"QdgPWhjW[zM.6Fa75.]C s 7c3K z5wdy!]S40?7muUGhDAE1jDXB73u"J2)(fo 

-------------------------------------------------------------------------------------------------------------------


 After 50000 iterations

 Home:]  
As chour auld mus. everythings ycree of rufclass crouse cr..  
[Aver surringst]  
[Brioh:]  
Sbughtare  
Brey I'm to sem  
]tyU:]  
Gnow shell sire mi caric-,  
And everyth:]  
The Verlicks  
[Mrss yer:]

, HHes.  
[Eddee,  
Masss, Smok crielmide -f it endees  
Surss Me love extlas so have mest whine fleide me him ofry